<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#libraries-and-widgets" data-toc-modified-id="libraries-and-widgets-0.0.0.1"><span class="toc-item-num">0.0.0.1&nbsp;&nbsp;</span>libraries and widgets</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Neuromatch-Academy-Project---Steinmetz-Dataset---Mouse-Mind-Reading" data-toc-modified-id="Neuromatch-Academy-Project---Steinmetz-Dataset---Mouse-Mind-Reading-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Neuromatch Academy Project - Steinmetz Dataset - Mouse Mind Reading</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#explore-dictionary-for-session" data-toc-modified-id="explore-dictionary-for-session-1.0.0.1"><span class="toc-item-num">1.0.0.1&nbsp;&nbsp;</span>explore dictionary for session</a></span></li></ul></li></ul></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load Data</a></span><ul class="toc-item"><li><span><a href="#via-google-drive-directly" data-toc-modified-id="via-google-drive-directly-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>via google drive directly</a></span></li><li><span><a href="#cleanup-channel-infos" data-toc-modified-id="cleanup-channel-infos-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>cleanup channel infos</a></span></li><li><span><a href="#cleaning-up-spikes" data-toc-modified-id="cleaning-up-spikes-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>cleaning up spikes</a></span></li><li><span><a href="#cleanup-trial-info" data-toc-modified-id="cleanup-trial-info-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>cleanup trial info</a></span></li></ul></li></ul></li></ul></div>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import matplotlib.pyplot as plt
import qgrid

from nma_class import NMA_project

%matplotlib inline

#### libraries and widgets
- qgrid
- nbextentions

In [2]:
# only if you are on jupyter notebook not if on colab
import qgrid

In [3]:
# only if you are on google coalb not if you are on jupyter notebook
#from google.colab import drive
#drive.mount('/content/gdrive')

# Neuromatch Academy Project - Steinmetz Dataset - Mouse Mind Reading

In [4]:
# link to parent folder with all datasets
#main_folder = "/content/gdrive/My Drive/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"
#main_folder = "C:/Users/User/Google Drive/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"
#main_folder = "Z:/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"
main_folder = os.path.join('//Volumes','Samsung_T5','steinmetz_dataset')

# create session object
nma_obj = NMA_project(main_folder) 

# default session is initialized, extract interesting dataframes in dedicated variables
session = nma_obj.std_session
channels_df = session['channels_df']
spikes_df = session['spikes_df']
clusters_df = session['clusters_df']
trials_df = session['trials_df']


In [6]:
channels_df

,ccf_ap,ccf_dv,ccf_lr,allen_ontology,channel 0,channel 1,probe,raw row,channels_site
0,4432.3,4699.7,5002.3,LS,43.0,20.0,0.0,0,0
1,4428.4,4699.7,4970.6,LS,11.0,20.0,0.0,1,1
2,4431.3,4680.0,5018.6,LS,59.0,40.0,0.0,2,2
3,4427.3,4680.0,4986.8,LS,27.0,40.0,0.0,3,3
4,4426.3,4660.2,5003.1,LS,43.0,60.0,0.0,4,4
...,...,...,...,...,...,...,...,...,...
743,9587.1,1073.8,2484.3,VISp,59.0,3800.0,1.0,378,378
744,9601.2,1054.6,2493.6,root,43.0,3820.0,1.0,380,380
745,9620.7,1054.6,2519.0,root,11.0,3820.0,1.0,381,381
746,9595.8,1035.4,2477.6,root,59.0,3840.0,1.0,382,382


#### explore dictionary for session
- each ession is a dictionary
- each element kontains eather a numpy array or a dataframe
- the most important dataframes we are going to work with are:
    - clusters_df: all the spikes and clusters
    - trials_df: all the behaviour data for all trials

In [ ]:
for key, value in nma_obj.std_session.items():
    print(key, value.shape)

In [ ]:
qgrid.show_grid(channels_df)

In [ ]:
qgrid.show_grid(clusters_df)

In [ ]:
qgrid.show_grid(trials_df)

## Load Data


### via google drive directly

In [ ]:
def load_session(folder):
    """
    load all files from given folder to dict
    return dict with key as filenames
    """
    # load session files ===========================================
    from os import listdir
    from os.path import isfile, join
    files = [f for f in listdir(folder) if isfile(join(folder, f))]
    names = ['_'.join(f.split('.')[:-1]) for f in files]
    session = dict()
    for (file_, name_) in zip(files, names):
        if file_.split('.')[-1] == 'npy':
        session[name_] = np.load(folder+'/'+file_)
        if file_.split('.')[-1] == 'tsv':
        session[name_] = pd.read_table(folder+'/'+file_)

    # pars session info two dataframe ===============================
    # create spike_df tataframe, with each spike time and the cluster it belongs to
    spikes_df = pd.DataFrame( { 'cluster':session['spikes_clusters'][:,0], 'spike_times': session['spikes_times'][:,0] }, ) 

    # create cluster dataframe information and spikes for each cluster
    clusters = np.unique(session['spikes_clusters'])
    # create number of spikes, phy2 manual cluster, 
    # parse phy annotation
    clusters_df = pd.DataFrame({'lable':session['clusters__phy_annotation'][:,0], 'probe':session['clusters_probes'][:,0], 'peak channel':session['clusters_peakChannel'][:,0]}, index=clusters )
    # pars labels in strings 1 = good 3 = mua
    #clusters_df['lable'] = clusters_df['lable'].apply( lambda lable: 'good' if lable==3 else ('mua' if lable==1 else 'bad') )

    # create spikes colum with spiketimes
    spk = pd.DataFrame( {'spikes':np.zeros(len(clusters), dtype=object)}, index=clusters )
    for group, frame in spikes_df.groupby('cluster'):
        spk['spikes'][group] = frame['spike_times'].values

    #merge spike column with clusters_df
    clusters_df = pd.merge(clusters_df, spk, how='right', left_index=True, right_index=True)
    # set index name
    clusters_df.index.name='cluster'
    # number of spikes per cluster
    clusters_df['total spikes'] = clusters_df['spikes'].apply(lambda row: len(row) )

# def load_session(folder, sessions):
#   """
#   load all files in each given session and add to session dict
#   returs a dict of sessions, with each session a dict of files
#   """
#   sessions_data = dict()
#   for session in sessions:
#     current_session = dict()
#     sessions[session] = 

In [ ]:
# example google colab load session
#folder = "/content/gdrive/My Drive/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"
#folder = "C:/Users/User/Google Drive/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"
folder = "Z:/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"

subfolders = [ f.path for f in os.scandir(folder) if f.is_dir() ]
sessions = [folder.split('/')[-1] for folder in subfolders]

# load first session
session = load_session(subfolders[0])

In [ ]:
folder = "Z:/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset/Cori_2016-12-14"
session = load_session(folder)

In [ ]:
for key in session.keys():
  print(key)

### cleanup channel infos

In [ ]:
# load channel brain location infos
brain = pd.DataFrame(session['channels_brainLocation'])
# load channel probes
site = pd.DataFrame(session['channels_sitePositions'], columns=['channel 0', 'channel 1'])
# merge with channles_df
channels_df = pd.merge(brain, site, how='inner', left_index=True, right_index=True)
# load probe, row and site
meta = pd.DataFrame({'probe':session['channels_probe'][:,0],
                            'raw row':session['channels_rawRow'][:,0],
                            'channels_site':session['channels_site'][:,0] })
# merge metha with channel_df
channels_df = pd.merge(channels_df, meta, how='inner', left_index=True, right_index=True)

In [ ]:
qgrid.show_grid(channels_df)

### cleaning up spikes

In [ ]:
# create spike_df tataframe, with each spike time and the cluster it belongs to
spikes_df = pd.DataFrame( { 'cluster':session['spikes_clusters'][:,0], 'spike_times': session['spikes_times'][:,0] }, ) 
qgrid.show_grid(spikes_df)

In [ ]:
# create cluster dataframe information and spikes for each cluster
clusters = np.unique(session['spikes_clusters'])
# create number of spikes, phy2 manual cluster, 
# parse phy annotation
clusters_df = pd.DataFrame({'lable':session['clusters__phy_annotation'][:,0], 'probe':session['clusters_probes'][:,0], 'peak channel':session['clusters_peakChannel'][:,0]}, index=clusters )
# pars labels in strings 1 = good 3 = mua
#clusters_df['lable'] = clusters_df['lable'].apply( lambda lable: 'good' if lable==3 else ('mua' if lable==1 else 'bad') )



# create spikes colum with spiketimes
spk = pd.DataFrame( {'spikes':np.zeros(len(clusters), dtype=object)}, index=clusters )
for group, frame in spikes_df.groupby('cluster'):
    spk['spikes'][group] = frame['spike_times'].values

#merge spike column with clusters_df
clusters_df = pd.merge(clusters_df, spk, how='right', left_index=True, right_index=True)
# set index name
clusters_df.index.name='cluster'
# number of spikes per cluster
clusters_df['total spikes'] = clusters_df['spikes'].apply(lambda row: len(row) )

In [ ]:
qgrid.show_grid(clusters_df)

### cleanup trial info

In [ ]:
print(session['trials_feedbackType'].shape)
print(session['trials_feedback_times'].shape)
print(session['trials_goCue_times'].shape)
print(session['trials_included'].shape)
print(session['trials_intervals'].shape)
print(session['trials_repNum'].shape)
print(session['trials_response_choice'].shape)
print(session['trials_response_times'].shape)
print(session['trials_visualStim_contrastLeft'].shape)
print(session['trials_visualStim_contrastRight'].shape)
print(session['trials_visualStim_times'].shape)

In [ ]:
# create trials dataframe
trials_df = pd.DataFrame({'included':session['trials_included'][:,0],
                          'repetition number':session['trials_repNum'][:,0],
                          # times of events
                          'go cue':session['trials_goCue_times'][:,0],
                          'stim time':session['trials_visualStim_times'][:,0],
                          'response time':session['trials_response_times'][:,0],
                          'feedback time':session['trials_feedback_times'][:,0],
                          
                          #'intervals':session['trials_intervals'][:,0], # same as go cue
                          
                          
                          'go cue':session['trials_intervals'][:,0],
                          
                          'stim contrast left':session['trials_visualStim_contrastLeft'][:,0],
                          'stim contrast right':session['trials_visualStim_contrastRight'][:,0],
                          
                          'response choice':session['trials_response_choice'][:,0],
                          'response time':session['trials_response_times'][:,0],
                          'feedback type':session['trials_feedbackType'][:,0],
                          
                         })

In [ ]:
qgrid.show_grid(trials_df)

In [ ]:
# add wheelmovement
print(session['wheel_position'].shape)
print(session['wheel_timestamps'].shape)
print(session['wheelMoves_intervals'].shape)
print(session['wheelMoves_type'].shape)

In [ ]:
wheel_df = pd.merge( pd.DataFrame(session['wheelMoves_intervals']), pd.DataFrame(session['wheelMoves_type']), how='left', left_index=True, right_index=True)

In [ ]:
qgrid.show_grid(pd.DataFrame(wheel_df))